# Language Models

В этот раз поработаем с более наглядными и чуть более интуитивными методами - `Bag of words` и вероятностной моделью.

In [ ]:
import numpy as np

with open('SpamDatas.txt', 'r') as f:
  full_text = f.read()

## Bag of words

Суть метода похожа на `word2vec`, но с одним отличием: раньше мы сопоставляли каждому слову вектор какой-то длины, а теперь будем сопоставлять вектор целому предложению.

Как правило, этим методом пользуются (пользовались, точнее), когда нужно быстро и интуитивно:
* Проверить похожесть предложений
* Классифицировать предложения (например, проверить на спам)
* Любые другие задачи, которые требуют обращать внимание на клюбчевые слова

На сегодня наша задача следующая: будем пытаться определить ботов.

Для начала, давайте посмотрим на имеющиеся данные:

In [ ]:
print(full_text[:993])

Давайте представим все в более читабельном виде.

In [ ]:
import pandas as pd

lines = [x.split('\t') for x in full_text.split('\n')]

df = pd.DataFrame(columns=['target', 'text'], data=lines)
df

Так-то лучше!

Итого, у нас есть 5574 смс-сообщения, для каждого из которых указано, спам это (`spam`), или нет (`ham`).

Как не странно, те, кто шлют спам - боты, именно их мы и будем стараться определить.

### Предложения, слова, снова предложения

Самая первая задача - посчитать общее количество уникальных слов, встречающихся в этих сообщениях - именно такой длины будет вектор для каждого предложения. Порядок действий будет такой:
1. Сначала, сообщения, конечно, нужно токенизировать - сложно смотреть на набор буквы, нам нужны слова! 
 
 Для этого вновь воспользуемся **nltk**.

2. Дальше, воспользуемся `Counter`'ом, чтобы узнать количество уникальных слов (или можно как-то еще?)
 
 **Не забудьте привести все к нижнему регистру, применить стемминг (`stemmer.stem()`) и убрать стоп-слова**

3. Создадим `base_dict` следующего вида:

 * Ключи - найденные раньше уникальные слова
 * Значения - `0` (он же базовый все-таки)

#### 1

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english") 
tokenizer = WordPunctTokenizer()

#### 2

In [ ]:
# TODO

total_words = ...

In [ ]:
total_words

#### 3

In [ ]:
# TODO

base_dict = ... 

In [ ]:
assert total_words == 7502, 'Неправильно посчитано количество слов'
assert len(base_dict) == total_words, 'Вектор учитывает не все слова'
assert sum(base_dict.values()) == 0, 'Значения `base_dict` должны быть нулями'

### Подсчет векторов для предложений

Теперь необходимо сделать следующее: построить вектор параметров и вектор истинных ответов для обучения.

Если с вектором истинных ответов есть лишь одна хотелка - сделать `spam -> 1`, `ham -> -1`, то с вектором параметров ситуация чуть сложнее, а именно:

1. Необходимо для каждого текста скопировать `base_dict`.
2. Далее, заполнить его так, чтобы у каждого `key` было `value`, соответствующее числу повторений этого `key` в текущем тексте.
3. Также необходимо создать `target_vector` - вектор целей (`1` или `-1`) в том же порядке, в котором добавлялись словари в предыдущем пункте.
3. Таким образом, должно получиться, что `i` элемент в `text_vectors` является словарем, и его классом является `i` элемент из `target_vector`

In [ ]:
from collections import Counter
from tqdm.notebook import tqdm

target2val = {'ham': -1, 'spam': 1}
# То есть 1 - это спам, а -1 - это нормальное сообщение

# TODO

text_vectors = ... 
target_vector = ...

In [ ]:
assert len(text_vectors[0]) == total_words \
        and len(text_vectors) == len(target_vector), 'Неправильно составлены вектора,'

In [ ]:
X = [list(x.values()) for x in text_vectors]
y = target_vector

X_train = X[:5000]
X_test = X[5000:]
y_train = y[:5000]
y_test = y[5000:]

### Предсказания

Итак, дело за малым - давайте построим логистическую регрессию, чтобы классифицировать спам и Не-спам.

p.s. если вы впервые слышите про логистическую регрессию - [тык](https://habr.com/ru/company/io/blog/265007/) сюда

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# TODO

model = ...

In [ ]:
model.score(X_test, y_test)

Теперь даже можно попробовать самому попридумывать спам, чтобы обмануть нашу, кажется, довольно не глупую модель.

In [ ]:
val2target = {-1 : 'ham', 1 : 'spam'}

def text_to_answer(text):
  cur_vector = base_dict.copy()
  for tok in tokenizer.tokenize(text):
    token = stemmer.stem(tok.lower())
    if token in stop_words:
      continue
    try:
      cur_vector[token] += 1
    except:
      print(token + ' not in vocabulary')

  return val2target[model.predict([list(cur_vector.values())])[0]]

In [ ]:
text_to_answer("You have won a 1 week FREE in SIRIUS! Txt the word: URA YA MOLODETS")

In [ ]:
text_to_answer('Your teacher is very happy that you translated this sentence :)')

In [ ]:
# Тут можно поиграться. А можно и не играться...

## N-граммы

Пришло время сделать маленький шаг в сторону темы курса - давайте попробуем что-то посочинять. К примеру, посочиняем спамовые сообщения!

### Модель N_gramm

На самом деле, сама логика (опять же) довольно проста. Мы будем генерировать слово, обращая внимание на `n` предыдущих слов - мы будем генерировать слова исходя из формулы 

$$
P(x_i \mid x_{i-n}, \dots, x_{i-1})
$$

Переведем ее на человеко-читаемый язык - если мы обучались на трех предложениях:

* "В детстве я любил гулять"
* "В детстве я любил, когда мама забирала меня из детского сада пораньше"
* "В детстве я любил преподавателей, которые не отправляли меня на пересдачу 4 раза в год"

тогда для `n=4`, `i=5`, у нас получается:

* $P('гулять' \mid 'В', 'детстве', 'я', 'любил') = \frac{1}{3}$
* $P(',' \mid 'В', 'детстве', 'я', 'любил') = \frac{1}{3}$
* $P('преподавателей' \mid 'В', 'детстве', 'я', 'любил') = \frac{1}{3}$

Почему так? Потому что в случае начала предложения со слова 'В', последующие 3 определятся однозначно, ведь так? И только на 5 слове у нас возникнет какая-то вероятность, и уже после этого мы с вероятностью $\frac{1}{3}$ выдадим слово `гулять`, `,` или `преподавателей`.

Абсолютно очевидно, что для генерации чего-то действительно интересного нам понадобится ооооочень большой датасет (хотя, он кажется всегда нужен).

Для решения этого довольно интересного задания предлагаем вам реализовать целый класс!

p.s. если вы не работали с классами, попросите преподавателя, он расскажет, что к чему :)

In [ ]:
class N_gramm:
  def __init__(self, n=3):
    # Метод инициализации класса - достаточно просто сохранить параметры
    self.n = n
  
  def fit(self, sentences):
    # Метод тренировки
    # На вход поступает список из списка токенов
    # Ваша задача - пройтись по этим спискам окном размера n и составить словарь token_probs:
    #   Его ключи - n-граммы
    #   Его значения - словари, у которых
    #     Ключи - токены, следующие за соответствующей n-граммой
    #     Значения - вероятность получения этого токена при условии соответствующей n-граммы
    # Таким образом, для упомянутого выше примера получим
    # token_probs['В детстве я любил']['гулять'] = 0.3333333333333333

    # TODO

    token_probs = ...
  
  def predict(self, prefix):
    # Метод по входному префиксу (тексту) предсказывает следующий токен.
    assert len(tokenizer.tokenize(prefix)) >= self.n, 'Префиксы должен быть длины хотя бы {} токена'.format(self.n)
    
    ngram = tokenizer.tokenize(prefix)[-self.n:]
    try:
      tokens = self.token_probs[' '.join(ngram)]
      next_token = np.random.choice(list(tokens.keys()), p=list(tokens.values()))
      return next_token, prefix + ' ' + next_token
    except:
      return None, None
  
  def generate_text(self, start_text, max_length=100, stop_prob = 0.2):
    # Используя метод predict вам необходимо реализовать метод generate_text, 
    # Который будет возвращать текст длины не больше чем max_length.
    # Для вероятности преждевременного завершения нужно после генерации одного из знаков 
    # '.', '?', '!' с вероятностью stop_prob завершать генерацию.

    # TODO

    generated_text = ...

    return generated_text

In [ ]:
ng = N_gramm(n=4)

ng.fit(["В детстве я любил гулять",
        "В детстве я любил, когда мама забирала меня из детского сада пораньше",
        "В детстве я любил преподавателей, которые не отправляли меня на пересдачу 4 раза в год"])

assert len(ng.token_probs) == 20, 'Неправильно работает fit'
assert np.isclose(np.mean([sum(x.values()) for x in ng.token_probs.values()]), 1), 'Вероятность продолжения каждой n-граммы долджна быть равна 1'

### Подготовка датасета

А теперь пора создать датасет.

Поскольку мы хотим научиться создавать спам-письма, будет логично на них и обучиться.

Задача следующая:

* Выбрать все текста, у которых `target=spam`
* Собрать их в один списоки и отправить обучаться в `N_gramm`.

In [ ]:
# TODO

spam_texts = ...

### Обучение спаму

In [ ]:
# TODO

NGramm = ...

In [ ]:
NGramm.generate_text('Hey there')

# Extra Task

Давайте на момент вспомним ту самую одноклассницу, которая прочитала все фанфики по Гарри Поттеру (мы уверены, у вас такая есть) и убедимся в том, что она прочитала НЕ ВСЕ.

А именно - давайте сами его напишем! Точнее, попросим нашу прикольную модельку его написать. 

Это бонусное задание, его делать совсем не обязательно. 

Ваша задача - обучить `N_gramm` на отрывке из Гарри Поттера из предыдущей домашки и попробовать за счет параметров получить какую-то +- осмысленную историю.